# Sentiment Analyse mit Transformers

Zielsetzung: In diesem Notebook verwenden wir einen Transformer zur Analyse unserer Daten. Basierend auf den Erkenntnissen aus dem vorherigen Kapitel sollte dieser theoretisch eine schlechtere Performance aufweisen als das Bag-of-Words-Modell aus den ersten Notebooks, jedoch besser abschneiden als das LSTM-Modell.

Alle Imports:

In [111]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
from tensorflow import keras
from tensorflow.keras import layers


## Reduzierte Ausführungszeit durch lokale Ausführung
Da nicht jeder über eine leistungsstarke Grafikkarte mit GPU-Unterstützung verfügt, kann die Ausführung von LSTMs zeitaufwändig sein. Eine effektive Alternative zur lokalen Ausführung bietet Google Colab. Dieser Dienst ermöglicht die kostenlose Ausführung des Codes mit GPU-Unterstützung. Obwohl die Authentifizierung bei Diensten wie Kaggle etwas komplexer sein kann, haben wir in den Notebooks Tools integriert, die eine einfache Authentifizierung in Google Colab ermöglichen.

## Authentifizierung bei Kaggle
Navigieren Sie zu https://www.kaggle.com. Gehen Sie dann zu Ihrem [Benutzerprofils](https://www.kaggle.com/me/account) und wählen Sie "API-Token erstellen" aus. Dadurch wird die Datei kaggle.json heruntergeladen, die Ihre API-Zugangsdaten enthalten.

Führen Sie anschließend die nachstehende Zelle aus, um kaggle.json in Ihrer Colab-Laufzeit hochzuladen.

In [112]:
from google.colab import files
    
uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
            name=fn, length=len(uploaded[fn])))

# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

ModuleNotFoundError: No module named 'google.colab'

## Herunterladen der Daten

In [113]:
import os

# Download McDonalds dataset
if not os.path.exists('mcdonalds-store-reviews.zip'):
    print("Downloading McDonalds dataset...")
    !kaggle datasets download -d nelgiriyewithana/mcdonalds-store-reviews
if os.path.exists('mcdonalds-store-reviews.zip'):
    print("Unzipping McDonalds dataset...")
    !unzip -n mcdonalds-store-reviews.zip

# Download IMDB dataset
if not os.path.exists('imdb-dataset-of-50k-movie-reviews.zip'):
    print("Downloading IMDB dataset...")
    !kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
if os.path.exists('imdb-dataset-of-50k-movie-reviews.zip'):
    print("Unzipping IMDB dataset...")
    !unzip -n imdb-dataset-of-50k-movie-reviews.zip

Unzipping McDonalds dataset...
66868.14s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
Archive:  mcdonalds-store-reviews.zip
Unzipping IMDB dataset...
66873.29s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
Archive:  imdb-dataset-of-50k-movie-reviews.zip


## Prozessierung der Daten
Die Prozessierung wird nicht weiterbeschrieben, weil sie identisch zum letzten Notebook ist. 

In [114]:
df_mc = pd.read_csv('McDonald_s_Reviews.csv', encoding="latin-1")
df_imdb = pd.read_csv('IMDB Dataset.csv')
df_mc = df_mc[df_mc['rating'] != '3 stars']
data_mc = df_mc['review'].to_numpy()
data_imdb = df_imdb['review'].to_numpy()
rating_mapping_imdb = {
    'positive': 1,
    'negative': 0,
}

label_imdb = df_imdb['sentiment'].map(rating_mapping_imdb).to_numpy()
rating_mapping_mc = {
    '1 star': 0,
    '2 stars': 0,
    '4 stars': 1,
    '5 stars': 1
}

label_mc = df_mc['rating'].map(rating_mapping_mc).to_numpy()
data = np.append(data_imdb, data_mc)
label = np.append(label_imdb,label_mc)

train_data, test_data, train_label, test_label = train_test_split(data, label, test_size=0.2, random_state=42)




## Wordembedding
Wir nutzen das gleiche Encoding für den Transformer wie schon im LSTM.

In [115]:
from tensorflow.keras import layers

max_length = 600
max_tokens = 20000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)
text_vectorization.adapt(train_data)
text_vectorization.adapt(test_data)

int_train_ds = text_vectorization(train_data)

int_test_ds = text_vectorization(test_data)

## Transformer-Encoder 
Self Attention:

Bei Self-Attention geht es darum, eine Beziehung zwischen verschiedenen Tokens innerhalb einer Sequenz herzustellen, um die wichtigsten Informationen zu identifizieren und zu betonen. 

Der Prozess der Self-Attention besteht aus drei grundlegenden Schritten:

1. Berechnung von Schlüssel-, Wert- und Abfragevektoren: Jedes Token in der Eingabesequenz wird in drei Vektoren transformiert - Schlüssel (Key), Wert (Value) und Abfrage (Query). 


2. Berechnung der Aufmerksamkeitsgewichte: Für jedes Token in der Sequenz werden Aufmerksamkeitsgewichte berechnet, um seine Beziehung zu anderen Tokens zu bestimmen. Dies geschieht, indem das Skalarprodukt zwischen dem Abfragevektor des aktuellen Tokens und den Schlüsselvektoren aller anderen Tokens berechnet wird. Durch die Anwendung einer Softmax-Funktion auf diese Skalarprodukte werden die Aufmerksamkeitsgewichte normalisiert.
> output = sum(values x pairwise-scores(query, keys))

3. Aggregation der Wertvektoren: Die Aufmerksamkeitsgewichte werden verwendet, um gewichtete Summen der Wertvektoren zu berechnen. Dies ermöglicht die Gewichtung der relevanten Informationen jedes Tokens entsprechend den Aufmerksamkeitsgewichten. Das Ergebnis ist der aggregierte Ausgabevektor für das aktuelle Token.

## Was sind die Query, Keys and Values in unserem Model?
In unserem Sequence-to-Sequence-Modell sind die Query, Keys und Values alles die gleichen inputs. Sie repräsentieren die Sequenz selbst, die mit sich selbst verglichen wird, um jedes Token mit Kontextinformationen aus der gesamten Sequenz anzureichern. In diesem Fall werden die Query-, Keys- und Values-Vektoren verwendet, um den Self-Attention-Mechanismus anzuwenden und wichtige Beziehungen und Zusammenhänge innerhalb der Sequenz zu erfassen.

<img src="assets/ScaledDotProduct.jpeg" alt="Multi Head Attention" width="200"/>ee


## Multi-Head Attention 
Der "Multi-head attention" Layer wurde in dem Paper "Attention is all you need" eingeführt. Der Begriff "multi-head" bezieht sich darauf, dass der Self-Attention Layer in eine Reihe unabhängiger Teilräume aufgeteilt wird. Diese lernen unabhängig voneinander. 

Die Query, Key und Value Vektoren werden durch drei separate Layern von dichten Projektionen geschickt, was zu drei separaten Vektoren führt. Jeder Vektor wird mit Hilfe von Self Attention verarbeitet, und die drei Ausgaben werden wieder zu einem einzelnen Output zusammengefügt. Jeder solche Teilraum wird als "head" bezeichnet.

Die dense Layers ermöglichen es der Gesamtschicht wirklich etwas zu lernen, ansonsten würde es sich nur um eine zustandslose Transformation handeln. Darüber hinaus hilft die Verwendung unabhängiger Heads der Schicht dabei, verschiedene Gruppen von Merkmalen für jedes Token zu lernen, wobei die Merkmale innerhalb einer Gruppe miteinander korreliert sind, aber größtenteils unabhängig von Merkmalen in einer anderen Gruppe sind.



<img src="assets/MultiHead.png" alt="Multi Head Attention" width="500"/>


## Transformer-Encoder 
Ein Transformer besteht aus einem Encoder und einem Decoder. Für unsere Sentiment-Analyse verwenden wir nur den Encoder, da wir keine neuen Sequenzen generieren müssen. Der Kern des Encoders ist der Multi-Head-Attention-Layer, der gerade erläutert wurde. Zusätzlich enthält der Encoder zusätzliche Dense-Layer, um weitere Beziehungen zwischen den Tokens zu erfassen. Residualverbindungen werden verwendet, um sicherzustellen, dass die Dense-Layer keine Informationen zerstören, und Normalisierungsschichten tragen dazu bei, die Daten auf zum Beispiel einen bestimmten Mittelwert zu normalisieren, was das Training stabiler und schneller macht.

<img src="assets/TransformerEncoder.png" alt="Transformer Encoder" width="300"/>


In [116]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

## Self Attention Model mit Wordembedding


In [117]:
# Hyperparameter
vocab_size = 20000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32


In [118]:
def getTransformerEncoderModel():
    inputs = keras.Input(shape=(None,), dtype="int64")
    x = layers.Embedding(vocab_size, embed_dim)(inputs)
    x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
    x = layers.GlobalMaxPooling1D()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
    return model

encoderModel = getTransformerEncoderModel()
encoderModel.summary()

Model: "model_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_35 (InputLayer)       [(None, None)]            0         
                                                                 
 embedding_28 (Embedding)    (None, None, 256)         5120000   
                                                                 
 transformer_encoder_15 (Tra  (None, None, 256)        543776    
 nsformerEncoder)                                                
                                                                 
 global_max_pooling1d_14 (Gl  (None, 256)              0         
 obalMaxPooling1D)                                               
                                                                 
 dropout_14 (Dropout)        (None, 256)               0         
                                                                 
 dense_46 (Dense)            (None, 1)                 257

Der Großteil der Komplexität dieses Modells liegt im Transformer Encoder. Wir müssen lediglich die Rückgabewerte des Encoders umwandeln, da er nur vollständige Sequenzen zurückgibt. Wir benötigen jedoch einzelne Vektoren, um die Daten zu klassifizieren. Daher nutzen wir einen GlobalMaxPooling1D Layer, um die Daten zu reduzieren.

In [119]:
callbacks = [
  keras.callbacks.EarlyStopping(
        monitor="val_loss",  # Metric to monitor
        patience=3,  # Number of epochs with no improvement after which training will be stopped
        restore_best_weights=True,  # Restore the weights of the best epoch
    )
]

In [120]:

encoderModel.fit(int_train_ds, train_label, validation_split=0.2, epochs=20, callbacks=callbacks)

Epoch 1/20
   5/1572 [..............................] - ETA: 21:42 - loss: 2.3241 - accuracy: 0.4688

KeyboardInterrupt: 

In [121]:
encoderModel.evaluate(int_test_ds, test_label)

  5/492 [..............................] - ETA: 2:38 - loss: 0.7225 - accuracy: 0.5437

KeyboardInterrupt: 

## Hinzufügen von PositionalEmbedding
Die Ergebnisse mit unserem Transformer-Layer sind bereits vielversprechend. Allerdings nutzen wir bisher nur einen der beiden wesentlichen Bestandteile, die den Transformer so leistungsstark machen: den Self Attention Mechanismus. Bisher hatte unser Modell keine Kenntnis über die Reihenfolge der Wörter, obwohl diese einen großen Unterschied in der Bedeutung des Satzes ausmachen können. Um dies zu verbessern, werden wir das Positional Embedding hinzufügen, um die Reihenfolge der Wörter zu berücksichtigen und so die Leistung unseres Modells weiter zu steigern.

Der aktuelle Transformer würde diese beiden Sätze identisch bewerten:

> "Eventough I did _**not**_ like the new mayo formula I was satisfied"

> "Eventough I did like the new mayo formula I was _**not**_ satisfied"

Ein Ansatz zur Integration von positionellen Daten besteht darin, die Daten sequenziell zu verarbeiten. Dieser Ansatz wird auch von LSTMs verwendet, um positionelle Daten in ihre Modelle einzufügen. Es ist jedoch wichtig zu beachten, dass dieser sequenzielle Ansatz keine parallele Verarbeitung ermöglicht. 

Im Paper "Attention is all you need" wird das Problem der Positionsdaten durch die Anreicherung des Word Embedding Vektors mit positionellen Informationen gelöst. Dieser Ansatz besteht darin, Sinus- und Kosinuswellenfrequenzen zu verwenden, um die Positionsdaten zu kodieren.

Anstatt separate Positional Embeddings zu verwenden oder die Daten sequenziell zu verarbeiten, fügt das Paper die Positionsdaten direkt in den Word Embedding Vektor ein. Hierzu werden Sinus- und Kosinusfunktionen mit unterschiedlichen Frequenzen verwendet, um die Positionsinformationen zu kodieren. Diese Sinus- und Kosinuswerte werden dann mit den Word Embedding Vektoren addiert.

Durch die Verwendung von Sinus- und Kosinusfrequenzen in den Word Embedding Vektoren kann der Transformer-Modellarchitektur die Positionsinformationen der Wörter erfassen, ohne dass eine sequenzielle Verarbeitung oder separate Positional Embeddings erforderlich sind.


Im Bild und der Grafik unter diesem Text wird dies systematisch erklärt. Als Beispiel nehmen wir den Satz:

 > Die Prüfung war schwer
 
  als unseren Input.

Schauen wir uns das Wort "Prüfung" genauer an. Zuerst wird ihm ein Index aus dem Wörterbuch zugewiesen. Anschließend wird ein Word Embedding Vektor generiert. In diesem Beispiel werden nur 5 Dimensionen genutzt, während es in der Realität oft deutlich mehr sind. Im Paper "Attention is all you need" werden beispielsweise 512 Dimensionen verwendet. Für das Positional Embedding wird ein Vektor mit derselben Größe generiert wie für das Word Embedding.

Die Daten in diesem Vektor werden mithilfe der Wortposition (2), der Anzahl der Dimensionen (5) und des Index (0-4) ermittelt. Durch die Indexierung kann das Embedding über verschiedene Frequenzen hinweg die Wortposition vergleichen. Wie in der Grafik zu sehen ist, können zwei Wörter eine ähnliche Frequenz haben, wie zum Beispiel P0 und P2 bei i = 4.

Wenn das Modell nur i = 4 verwenden würde, um die Position zu ermitteln, hätte es Schwierigkeiten, die richtige Wortreihenfolge aus den Daten abzuleiten. Glücklicherweise besitzt es jedoch noch weitere Dimensionen, wie z.B. i = 2. In der Grafik können wir sehen, dass sich die Frequenz von P0 und P2 bei i = 2 deutlich unterscheiden.


<img src="assets/PositonalEmbedding.jpeg" alt="PositionalEncoding" width="1000"/>


<img src="assets/Frequenzen.png" alt="PositionalEncoding" width="500"/>


In [122]:
class PositionEmbeddingSinCos(layers.Layer):
    def __init__(self, sequence_length, vocab_size, output_dim, **kwargs):
        super(PositionEmbeddingSinCos, self).__init__(**kwargs)
        position_embedding_matrix = self.get_position_encoding(sequence_length, output_dim)                                          
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=output_dim)
        self.position_embedding_layer = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim,
            weights=[position_embedding_matrix]            )
             
    def get_position_encoding(self, seq_len, d, n=10000):
        P = np.zeros((seq_len, d))
        for k in range(seq_len):
            for i in np.arange(int(d/2)):
                denominator = np.power(n, 2*i/d)
                P[k, 2*i] = np.sin(k/denominator)
                P[k, 2*i+1] = np.cos(k/denominator)
        return P
 
    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

 
    def call(self, inputs):        
        position_indices = tf.range(tf.shape(inputs)[-1])
        embedded_tokens = self.token_embeddings(inputs)
        embedded_indices = self.position_embedding_layer(position_indices)
        return embedded_tokens + embedded_indices

In [123]:
def getPositonalEmbeddingModel():
    inputs = keras.Input(shape=(None,), dtype="int64")
    x = PositionEmbeddingSinCos(sequence_length,vocab_size,embed_dim)(inputs)
    x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
    x = layers.GlobalMaxPooling1D()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
    return model
positonalModel = getPositonalEmbeddingModel()
positonalModel.summary()

Model: "model_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_36 (InputLayer)       [(None, None)]            0         
                                                                 
 position_embedding_sin_cos_  (None, None, 256)        5273600   
 1 (PositionEmbeddingSinCos)                                     
                                                                 
 transformer_encoder_16 (Tra  (None, None, 256)        543776    
 nsformerEncoder)                                                
                                                                 
 global_max_pooling1d_15 (Gl  (None, 256)              0         
 obalMaxPooling1D)                                               
                                                                 
 dropout_15 (Dropout)        (None, 256)               0         
                                                          

In [125]:
positonalModel.fit(int_train_ds,train_label, validation_split=0.2, epochs=20, callbacks=callbacks)


Epoch 1/20
   3/1572 [..............................] - ETA: 22:20 - loss: 2.2154 - accuracy: 0.5833 

KeyboardInterrupt: 

In [124]:
positonalModel.evaluate(int_test_ds, test_label)

  2/492 [..............................] - ETA: 2:57 - loss: 0.7404 - accuracy: 0.5469

KeyboardInterrupt: 

## Simpleres Positional Encoding
Bevor wir das Positional Embedding nach der "Attention is all you need"-Methode implementiert haben, haben wir ein einfacheres Positional Embedding verwendet. Dabei haben wir den Wordindex direkt als Positional Embedding Vektor erstellt und mit dem Word Embedding Vektor addiert. Überraschenderweise erzielte dieser einfache Ansatz die beste Performance unter allen drei Transformer-Modellen. Obwohl wir keine konkrete Erklärung dafür haben, könnten möglicherweise die Hyperparameter besser zu diesem Ansatz passen oder die Größe des Vokabulars (vocab_size) von 600 hier eine Rolle spielen. Wir wären gespannt auf ihre Meinung zu dieser Auswirkung!

In [126]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

In [127]:
def getSimplePosModel():
    inputs = keras.Input(shape=(None,), dtype="int64")
    x = PositionalEmbedding(sequence_length,vocab_size,embed_dim)(inputs)
    x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
    x = layers.GlobalMaxPooling1D()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
    return model
simplePosModel = getSimplePosModel() 
simplePosModel.summary()

Model: "model_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_37 (InputLayer)       [(None, None)]            0         
                                                                 
 positional_embedding_7 (Pos  (None, None, 256)        5273600   
 itionalEmbedding)                                               
                                                                 
 transformer_encoder_17 (Tra  (None, None, 256)        543776    
 nsformerEncoder)                                                
                                                                 
 global_max_pooling1d_16 (Gl  (None, 256)              0         
 obalMaxPooling1D)                                               
                                                                 
 dropout_16 (Dropout)        (None, 256)               0         
                                                          

In [128]:

simplePosModel.fit(int_train_ds,train_label, validation_split=0.2, epochs=20, callbacks=callbacks)


Epoch 1/20
   4/1572 [..............................] - ETA: 22:58 - loss: 1.9327 - accuracy: 0.5312

KeyboardInterrupt: 

In [129]:
simplePosModel.evaluate(int_test_ds, test_label)

  6/492 [..............................] - ETA: 2:55 - loss: 0.8618 - accuracy: 0.5417

KeyboardInterrupt: 

# Fazit
Wie im LSTM-Notebook bereits festgestellt wurde, sind Transformer nicht unbedingt die ideale Wahl für die Menge der vorliegenden Daten. Obwohl die Ergebnisse des Transformer-Modells minimal besser sind als die des LSTM-Modells, liegen sie nur fast auf dem Niveau der Bag-of-Words-Ansätze. Allerdings ist die Ausführungszeit des Transformer-Modells um ein Vielfaches höher. Wenn wir die Ausführungszeit und die noch vorhandenen Möglichkeiten der Hyperparameter-Optimierung im Bag-of-Words-Ansatz berücksichtigen, überwiegen die Vorteile der Bag-of-Words-Methodik deutlich.

Jedoch, wenn wir beabsichtigen, ein allgemeines Modell zu erstellen, das eine deutlich größere Anzahl von Reviews verarbeiten kann, dann ist die Transformer-Architektur die richtige Wahl. Insbesondere der Attention-Mechanismus und die parallel ausführbaren Positional Encodings machen sie in diesem Fall zur besten Wahl.